Create the respective folders first

In [1]:
import os
import yaml
import glob
import cv2

HOME = os.getcwd()

path = "/home/jovyan/yolov5/transfer_learning/spectrogram_v3.yaml"
with open(path, "r") as stream:
    try:
        classes = yaml.safe_load(stream)['names']
    except yaml.YAMLError as exc:
        print(exc)

delete = False        

for i in classes:
    try:
        os.mkdir(classes[i])
    except FileExistsError as e:
        # Delete everything, ie restart the whole thing
        if delete:
            for j in glob.glob(f"{HOME}/{classes[i]}/*"):
                os.remove(j)

Now I will read the files in "dataset1" and extract the respective signals into the individual folders

In [2]:
IMAGES_PATH = '/home/jovyan/spectrogram_training_data_20221006/dataset1/images'
LABELS_PATH = '/home/jovyan/spectrogram_training_data_20221006/dataset1/labels'

to_run = False


if to_run:
    for image in glob.glob(IMAGES_PATH + "/*.png")[:]: # Keep in mind that there are 20k images
        index = image.replace(IMAGES_PATH, "")
        index = index.replace(".png", "")
        label_path = LABELS_PATH + index + ".txt"
        img = cv2.imread(image)
        # print(img.shape)
        with open(label_path, "r") as r:
            lines = r.readlines()
            for index, line in enumerate(lines):
                label, x, y, w, h = line.split(" ")
                label = int(label)
                x = float(x)
                y = float(y)
                w = float(w)
                h = float(h)
                # print(label, x, y, w, h)
                y1 = (y-h/2)*img.shape[0]
                y2 = (y+h/2)*img.shape[0]
                x1 = (x-w/2)*img.shape[1]
                x2 = (x+w/2)*img.shape[1]
                counter_files = len(glob.glob(f"{HOME}/{classes[label]}/*"))
                final_img = img[int(y1): int(y2),int(x1):int(x2),:]
                try:
                    cv2.imwrite(f"{HOME}/{classes[label]}/{counter_files}.png", final_img)
                except:
                    print(classes[label], final_img.shape)

Start Analysis

In [3]:
folders = [f"{HOME}/{classes[c]}" for c in classes]
folders

['/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WAC_1',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WAC_3',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WAC_7',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WN_0',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WN_1',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WN_2',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WN_3',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WN_7',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WBG_0',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WBG_1',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WBG_2',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WBG_3',
 '/home/jovyan/spectrogram_training_data_20221006/signals/WLAN_WBG_7',
 '/home/jovyan/spectrogram_training_data_20221006/signals/BT_classic_ADH5',
 '/hom

In [4]:
largest = 0
for key in classes:
    print(classes[key], len(glob.glob(folders[key] + '/*')))
    if len(glob.glob(folders[key] + '/*')) > largest:
        largest = len(glob.glob(folders[key] + '/*'))

print(largest)

WLAN_WAC_1 15618
WLAN_WAC_3 27118
WLAN_WAC_7 47323
WLAN_WN_0 3347
WLAN_WN_1 15652
WLAN_WN_2 3523
WLAN_WN_3 17788
WLAN_WN_7 20244
WLAN_WBG_0 4111
WLAN_WBG_1 27443
WLAN_WBG_2 3511
WLAN_WBG_3 26159
WLAN_WBG_7 22557
BT_classic_ADH5 7381
BT_classic_AEDH5 7059
BLE_2MHz_DATA 17441
BLE_1MHz_DATA 8387
BLE_1MHz_AIND 3615
47323


Extract the Height and Width of each signal

In [6]:
import pandas as pd
import numpy as np

heightarray = np.zeros((47323,len(classes)))
widtharray = np.zeros((47323,len(classes)))
for x_index, signal_type in enumerate(folders):
    print(f"Progress ---- {x_index + 1}/18")
    for signal in glob.glob(signal_type + '/*'):
        index = int(signal.replace(signal_type+'/', "")[:-4])
        img = cv2.imread(signal)
        heightarray[index][x_index] = img.shape[0]
        widtharray[index][x_index] = img.shape[1]

df_height = pd.DataFrame(heightarray, columns = list(classes.values()))
df_height.to_csv('height.csv')
df_width = pd.DataFrame(widtharray, columns = list(classes.values()))
df_width.to_csv('width.csv')

Progress ---- 1/18
Progress ---- 2/18
Progress ---- 3/18
Progress ---- 4/18
Progress ---- 5/18
Progress ---- 6/18
Progress ---- 7/18
Progress ---- 8/18
Progress ---- 9/18
Progress ---- 10/18
Progress ---- 11/18
Progress ---- 12/18
Progress ---- 13/18
Progress ---- 14/18
Progress ---- 15/18
Progress ---- 16/18
Progress ---- 17/18
Progress ---- 18/18


In [9]:
data_height = pd.read_csv('height.csv', index_col = 0)
data_height.replace(0, np.nan, inplace=True)
data_height.mean()

WLAN_WAC_1          113.596747
WLAN_WAC_3          112.881555
WLAN_WAC_7          112.398686
WLAN_WN_0            72.173887
WLAN_WN_1            76.105098
WLAN_WN_2            68.856656
WLAN_WN_3            77.594895
WLAN_WN_7            79.394438
WLAN_WBG_0           70.821211
WLAN_WBG_1           70.391102
WLAN_WBG_2           74.094845
WLAN_WBG_3           70.607324
WLAN_WBG_7           69.624861
BT_classic_ADH5       4.181276
BT_classic_AEDH5      4.062048
BLE_2MHz_DATA         8.516255
BLE_1MHz_DATA         3.883152
BLE_1MHz_AIND         4.016044
dtype: float64

In [10]:
data_height.std()

WLAN_WAC_1          53.247454
WLAN_WAC_3          54.033523
WLAN_WAC_7          54.770722
WLAN_WN_0           46.819530
WLAN_WN_1           48.251483
WLAN_WN_2           47.773768
WLAN_WN_3           48.329174
WLAN_WN_7           48.570668
WLAN_WBG_0          46.888052
WLAN_WBG_1          47.311128
WLAN_WBG_2          47.007799
WLAN_WBG_3          47.209080
WLAN_WBG_7          47.045141
BT_classic_ADH5      2.139367
BT_classic_AEDH5     2.167890
BLE_2MHz_DATA        4.144118
BLE_1MHz_DATA        2.108535
BLE_1MHz_AIND        2.141060
dtype: float64

In [11]:
data_height.std()/data_height.mean()

WLAN_WAC_1          0.468741
WLAN_WAC_3          0.478675
WLAN_WAC_7          0.487290
WLAN_WN_0           0.648705
WLAN_WN_1           0.634011
WLAN_WN_2           0.693815
WLAN_WN_3           0.622840
WLAN_WN_7           0.611764
WLAN_WBG_0          0.662062
WLAN_WBG_1          0.672118
WLAN_WBG_2          0.634427
WLAN_WBG_3          0.668614
WLAN_WBG_7          0.675695
BT_classic_ADH5     0.511654
BT_classic_AEDH5    0.533694
BLE_2MHz_DATA       0.486613
BLE_1MHz_DATA       0.542996
BLE_1MHz_AIND       0.533127
dtype: float64

In [12]:
data_width = pd.read_csv('width.csv', index_col = 0)
data_width.replace(0, np.nan, inplace=True)
data_width.mean()

WLAN_WAC_1          103.296005
WLAN_WAC_3           59.027325
WLAN_WAC_7           33.093210
WLAN_WN_0           477.073797
WLAN_WN_1            58.961794
WLAN_WN_2           224.875390
WLAN_WN_3            36.422869
WLAN_WN_7            22.532207
WLAN_WBG_0          283.836779
WLAN_WBG_1           82.111978
WLAN_WBG_2          535.362290
WLAN_WBG_3          101.369242
WLAN_WBG_7           63.235271
BT_classic_ADH5      30.385856
BT_classic_AEDH5     70.153138
BLE_2MHz_DATA        45.953156
BLE_1MHz_DATA       144.319661
BLE_1MHz_AIND        51.896819
dtype: float64

In [13]:
data_width.std()

WLAN_WAC_1          200.515260
WLAN_WAC_3          103.319785
WLAN_WAC_7           45.532946
WLAN_WN_0           332.891689
WLAN_WN_1            93.955668
WLAN_WN_2           111.073440
WLAN_WN_3            49.205411
WLAN_WN_7            22.101934
WLAN_WBG_0          310.172563
WLAN_WBG_1           91.416327
WLAN_WBG_2          362.821615
WLAN_WBG_3          126.828238
WLAN_WBG_7           41.036698
BT_classic_ADH5      27.638494
BT_classic_AEDH5    117.564065
BLE_2MHz_DATA        59.094328
BLE_1MHz_DATA       182.270737
BLE_1MHz_AIND        26.408965
dtype: float64

In [14]:
data_width.std()/data_width.mean()

WLAN_WAC_1          1.941172
WLAN_WAC_3          1.750372
WLAN_WAC_7          1.375900
WLAN_WN_0           0.697778
WLAN_WN_1           1.593501
WLAN_WN_2           0.493933
WLAN_WN_3           1.350948
WLAN_WN_7           0.980904
WLAN_WBG_0          1.092785
WLAN_WBG_1          1.113313
WLAN_WBG_2          0.677712
WLAN_WBG_3          1.251151
WLAN_WBG_7          0.648953
BT_classic_ADH5     0.909584
BT_classic_AEDH5    1.675820
BLE_2MHz_DATA       1.285969
BLE_1MHz_DATA       1.262965
BLE_1MHz_AIND       0.508874
dtype: float64

In [15]:
data_width

,WLAN_WAC_1,WLAN_WAC_3,WLAN_WAC_7,WLAN_WN_0,WLAN_WN_1,WLAN_WN_2,WLAN_WN_3,WLAN_WN_7,WLAN_WBG_0,WLAN_WBG_1,WLAN_WBG_2,WLAN_WBG_3,WLAN_WBG_7,BT_classic_ADH5,BT_classic_AEDH5,BLE_2MHz_DATA,BLE_1MHz_DATA,BLE_1MHz_AIND
0,243.0,33.0,23.0,353.0,14.0,518.0,6.0,15.0,499.0,55.0,945.0,53.0,41.0,8.0,30.0,10.0,19.0,61.0
1,10.0,10.0,13.0,41.0,44.0,335.0,9.0,12.0,835.0,47.0,544.0,56.0,118.0,31.0,23.0,10.0,82.0,26.0
2,59.0,94.0,11.0,758.0,11.0,288.0,9.0,9.0,365.0,225.0,704.0,113.0,113.0,8.0,57.0,33.0,777.0,30.0
3,21.0,127.0,8.0,155.0,23.0,82.0,193.0,12.0,589.0,42.0,909.0,119.0,118.0,5.0,222.0,11.0,203.0,30.0
4,23.0,82.0,22.0,587.0,23.0,195.0,11.0,71.0,282.0,53.0,705.0,271.0,58.0,27.0,30.0,8.0,146.0,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47318,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47319,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47320,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47321,NaN,NaN,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data_height

,WLAN_WAC_1,WLAN_WAC_3,WLAN_WAC_7,WLAN_WN_0,WLAN_WN_1,WLAN_WN_2,WLAN_WN_3,WLAN_WN_7,WLAN_WBG_0,WLAN_WBG_1,WLAN_WBG_2,WLAN_WBG_3,WLAN_WBG_7,BT_classic_ADH5,BT_classic_AEDH5,BLE_2MHz_DATA,BLE_1MHz_DATA,BLE_1MHz_AIND
0,81.0,123.0,61.0,41.0,31.0,41.0,31.0,31.0,99.0,31.0,63.0,31.0,114.0,5.0,1.0,3.0,2.0,5.0
1,227.0,123.0,61.0,41.0,31.0,153.0,153.0,31.0,103.0,31.0,113.0,11.0,36.0,5.0,5.0,3.0,5.0,5.0
2,227.0,61.0,61.0,63.0,31.0,10.0,153.0,113.0,20.0,15.0,26.0,41.0,41.0,5.0,5.0,11.0,8.0,7.0
3,163.0,123.0,227.0,31.0,153.0,85.0,153.0,113.0,31.0,43.0,10.0,41.0,41.0,5.0,1.0,11.0,2.0,7.0
4,163.0,123.0,97.0,63.0,20.0,153.0,31.0,113.0,31.0,31.0,85.0,41.0,41.0,5.0,1.0,11.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47318,NaN,NaN,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47319,NaN,NaN,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47320,NaN,NaN,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47321,NaN,NaN,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data_area = data_width*data_height

In [19]:
data_area.mean()

WLAN_WAC_1          11844.648675
WLAN_WAC_3           6461.115938
WLAN_WAC_7           3544.835598
WLAN_WN_0           35118.786077
WLAN_WN_1            4172.641643
WLAN_WN_2           15284.426625
WLAN_WN_3            2576.836407
WLAN_WN_7            1671.893055
WLAN_WBG_0          20871.358793
WLAN_WBG_1           5717.635390
WLAN_WBG_2          40400.160068
WLAN_WBG_3           7080.436943
WLAN_WBG_7           4294.980184
BT_classic_ADH5       120.302263
BT_classic_AEDH5      261.554611
BLE_2MHz_DATA         360.449917
BLE_1MHz_DATA         525.646238
BLE_1MHz_AIND         202.284924
dtype: float64

In [20]:
data_area.std()

WLAN_WAC_1          25572.905577
WLAN_WAC_3          12745.532917
WLAN_WAC_7           5378.573576
WLAN_WN_0           33831.790908
WLAN_WN_1            8209.045091
WLAN_WN_2           12926.173255
WLAN_WN_3            4145.337439
WLAN_WN_7            2000.717091
WLAN_WBG_0          29121.895799
WLAN_WBG_1           8440.781765
WLAN_WBG_2          38124.476274
WLAN_WBG_3          11218.859597
WLAN_WBG_7           4181.016927
BT_classic_ADH5       123.409348
BT_classic_AEDH5      482.423742
BLE_2MHz_DATA         488.229640
BLE_1MHz_DATA         754.044636
BLE_1MHz_AIND         141.393396
dtype: float64

In [21]:
data_area.std()/data_area.mean()

WLAN_WAC_1          2.159026
WLAN_WAC_3          1.972652
WLAN_WAC_7          1.517298
WLAN_WN_0           0.963353
WLAN_WN_1           1.967350
WLAN_WN_2           0.845709
WLAN_WN_3           1.608693
WLAN_WN_7           1.196678
WLAN_WBG_0          1.395304
WLAN_WBG_1          1.476271
WLAN_WBG_2          0.943671
WLAN_WBG_3          1.584487
WLAN_WBG_7          0.973466
BT_classic_ADH5     1.025827
BT_classic_AEDH5    1.844447
BLE_2MHz_DATA       1.354501
BLE_1MHz_DATA       1.434510
BLE_1MHz_AIND       0.698981
dtype: float64

In [37]:
height = data_height.mean().to_frame(name='mean_h')
height_std = data_height.std().to_frame(name='std_h')
height_norm_std = (data_height.std()/data_height.mean()).to_frame(name='norm_std_h')
width = data_width.mean().to_frame(name='mean_w')
width_std = data_width.std().to_frame(name='std_w')
width_norm_std = (data_width.std()/data_width.mean()).to_frame(name='norm_std_w')

data_area = data_width*data_height

area = data_area.mean().to_frame(name='mean_a')
area_std = data_area.std().to_frame(name='std_a')
area_norm_std = (data_area.std()/data_area.mean()).to_frame(name='norm_std_a')

summary = pd.concat([height,height_std,height_norm_std,
                   width,width_std,width_norm_std,
                   area,area_std,area_norm_std], axis= 1)
summary

,mean_h,std_h,norm_std_h,mean_w,std_w,norm_std_w,mean_a,std_a,norm_std_a
WLAN_WAC_1,113.596747,53.247454,0.468741,103.296005,200.515260,1.941172,11844.648675,25572.905577,2.159026
WLAN_WAC_3,112.881555,54.033523,0.478675,59.027325,103.319785,1.750372,6461.115938,12745.532917,1.972652
WLAN_WAC_7,112.398686,54.770722,0.487290,33.093210,45.532946,1.375900,3544.835598,5378.573576,1.517298
WLAN_WN_0,72.173887,46.819530,0.648705,477.073797,332.891689,0.697778,35118.786077,33831.790908,0.963353
WLAN_WN_1,76.105098,48.251483,0.634011,58.961794,93.955668,1.593501,4172.641643,8209.045091,1.967350
WLAN_WN_2,68.856656,47.773768,0.693815,224.875390,111.073440,0.493933,15284.426625,12926.173255,0.845709
WLAN_WN_3,77.594895,48.329174,0.622840,36.422869,49.205411,1.350948,2576.836407,4145.337439,1.608693
WLAN_WN_7,79.394438,48.570668,0.611764,22.532207,22.101934,0.980904,1671.893055,2000.717091,1.196678
WLAN_WBG_0,70.821211,46.888052,0.662062,283.836779,310.172563,1.092785,20871.358793,29121.895799,1.395304
WLAN_WBG_1,70.391102,47.311128,0.672118,82.111978,91.416327,1.113313,5717.635390,8440.781765,1.476271


In [38]:
#exp11 Train is slicing == False mAP@0.5
mAP_no_slice = [0.472,
                0.534,
                0.687,
                0.796,
                0.549,
                0.674,
                0.472,
                0.537,
                0.719,
                0.421,
                0.747,
                0.395,
                0.316,
                0.376,
                0.246,
                0.724,
                0.312,
                0.433]
#exp9 Train is slicing == True mAP@0.5
mAP_slice = [0.737,
             0.820,
             0.917,
             0.711,
             0.800,
             0.498,
             0.874,
             0.914,
             0.633,
             0.425,
             0.583,
             0.403,
             0.296,
             0.580,
             0.461,
             0.878,
             0.467,
             0.580]

summary['mAP_slice'] = mAP_slice
summary['mAP_no_slice'] = mAP_no_slice
summary

,mean_h,std_h,norm_std_h,mean_w,std_w,norm_std_w,mean_a,std_a,norm_std_a,mAP_slice,mAP_no_slice
WLAN_WAC_1,113.596747,53.247454,0.468741,103.296005,200.515260,1.941172,11844.648675,25572.905577,2.159026,0.737,0.472
WLAN_WAC_3,112.881555,54.033523,0.478675,59.027325,103.319785,1.750372,6461.115938,12745.532917,1.972652,0.820,0.534
WLAN_WAC_7,112.398686,54.770722,0.487290,33.093210,45.532946,1.375900,3544.835598,5378.573576,1.517298,0.917,0.687
WLAN_WN_0,72.173887,46.819530,0.648705,477.073797,332.891689,0.697778,35118.786077,33831.790908,0.963353,0.711,0.796
WLAN_WN_1,76.105098,48.251483,0.634011,58.961794,93.955668,1.593501,4172.641643,8209.045091,1.967350,0.800,0.549
WLAN_WN_2,68.856656,47.773768,0.693815,224.875390,111.073440,0.493933,15284.426625,12926.173255,0.845709,0.498,0.674
WLAN_WN_3,77.594895,48.329174,0.622840,36.422869,49.205411,1.350948,2576.836407,4145.337439,1.608693,0.874,0.472
WLAN_WN_7,79.394438,48.570668,0.611764,22.532207,22.101934,0.980904,1671.893055,2000.717091,1.196678,0.914,0.537
WLAN_WBG_0,70.821211,46.888052,0.662062,283.836779,310.172563,1.092785,20871.358793,29121.895799,1.395304,0.633,0.719
WLAN_WBG_1,70.391102,47.311128,0.672118,82.111978,91.416327,1.113313,5717.635390,8440.781765,1.476271,0.425,0.421


In [39]:
summary.to_csv('summary.csv')

In [53]:
summary['mean_w']/data_width.max()

WLAN_WAC_1          0.047167
WLAN_WAC_3          0.053418
WLAN_WAC_7          0.072573
WLAN_WN_0           0.224929
WLAN_WN_1           0.064089
WLAN_WN_2           0.312327
WLAN_WN_3           0.077661
WLAN_WN_7           0.112661
WLAN_WBG_0          0.132325
WLAN_WBG_1          0.064911
WLAN_WBG_2          0.246824
WLAN_WBG_3          0.075932
WLAN_WBG_7          0.101015
BT_classic_ADH5     0.125045
BT_classic_AEDH5    0.066559
BLE_2MHz_DATA       0.089577
BLE_1MHz_DATA       0.067471
BLE_1MHz_AIND       0.279015
dtype: float64

In [54]:
data_width.max()

WLAN_WAC_1          2190.0
WLAN_WAC_3          1105.0
WLAN_WAC_7           456.0
WLAN_WN_0           2121.0
WLAN_WN_1            920.0
WLAN_WN_2            720.0
WLAN_WN_3            469.0
WLAN_WN_7            200.0
WLAN_WBG_0          2145.0
WLAN_WBG_1          1265.0
WLAN_WBG_2          2169.0
WLAN_WBG_3          1335.0
WLAN_WBG_7           626.0
BT_classic_ADH5      243.0
BT_classic_AEDH5    1054.0
BLE_2MHz_DATA        513.0
BLE_1MHz_DATA       2139.0
BLE_1MHz_AIND        186.0
dtype: float64

Extract Frames (20k images) information

In [48]:
to_run_frames = True

frames_array = np.zeros((20000, 2))

if to_run_frames:
    for index, image in enumerate(glob.glob(IMAGES_PATH + "/*.png")): # Keep in mind that there are 20k images
        img = cv2.imread(image)
        frames_array[index][0] = img.shape[0]
        frames_array[index][1] = img.shape[1]
        
df_frames = pd.DataFrame(frames_array, columns = ['height', 'width'])
df_frames.to_csv('frames_info.csv')
df_frames

,height,width
0,192.0,1024.0
1,192.0,1024.0
2,256.0,790.0
3,192.0,1024.0
4,192.0,1024.0
...,...,...
19995,256.0,1054.0
19996,192.0,1024.0
19997,192.0,1024.0
19998,256.0,790.0


In [52]:
set(df_frames['height'])

{128.0, 192.0, 256.0}

In [50]:
set(df_frames['width'])

{790.0, 878.0, 1024.0, 1054.0, 2196.0}